In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-13 19:56:46--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-13 19:56:46 (11.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [4]:
import os

In [5]:
import openai
import json

In [6]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [7]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

SELECT * WHERE course = 'data-engineering-zoomcamp';

In [11]:
q = 'the course has already started, can I still enroll?'

In [12]:
index.fit(documents)

In [20]:
from openai import OpenAI

In [21]:
client = OpenAI()

In [24]:
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"If the course has already started, it's best to check directly with the institution or organization offering the course. Policies on late enrollment can vary widely. Some courses might allow you to enroll a few days or even weeks after the start date, while others may not accept latecomers at all. Here are some steps you can take:\n\n1. **Contact the Instructor or Administration**: Reach out to the course instructor or the administrative office to inquire about the possibility of late enrollment.\n  \n2. **Check Online Portals**: If the course is offered online, there may be specific information or options available on the course website or learning management system.\n\n3. **Read the Course Policy**: Some courses have a syllabus or a course policy document that outlines the rules regarding late enrollment.\n\n4. **Consider Catching Up**: Be prepared to catch up on any missed material if you are allowed to enroll late.\n\n5. **Look for Alternatives**: If late enrollment is not allowed

In [26]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [27]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [28]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [29]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [30]:
rag(query)

"To run Kafka, follow the instructions based on your specific implementation:\n\n1. **For Java Kafka:**\n   If you're running a Java Kafka producer/consumer/KStreams, navigate to your project directory and use the following command:\n   ```sh\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n   Replace `<jar_name>` with the actual name of your JAR file.\n\n2. **For Python Kafka:**\n   If you're working with Python, it is recommended to create a virtual environment and install the required packages. Here’s how you can do it:\n   ```sh\n   python -m venv env\n   source env/bin/activate\n   pip install -r ../requirements.txt\n   ```\n   You will need to activate the virtual environment every time you run the Python script:\n   ```sh\n   source env/bin/activate\n   ```\n   To deactivate it, use:\n   ```sh\n   deactivate\n   ```\n   Note: For Windows, the activation command is slightly different:\n   ```sh\n   env\\Scripts\\activate

In [31]:
rag('the course has already started, can I still enroll?')

"Yes, even if the course has already started, you can still enroll and submit the homeworks. However, be mindful that there will be deadlines for the final projects, so don't leave everything until the last minute."

In [33]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [35]:
from elasticsearch import Elasticsearch

In [36]:
es_client = Elasticsearch('http://localhost:9200')

In [38]:
es_client.info()

ObjectApiResponse({'name': '7a3421da831d', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'kSKFqbH_SyKASkhv5w3hhA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [40]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [41]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:23<00:00, 40.39it/s]


In [44]:
query = 'I just disovered the course. Can I still join it?'

In [48]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [49]:
es_client.search(index=index_name, body=search_query)

NameError: name 'search_query' is not defined